In [31]:
from ollama import Client
from langchain_core.prompts import PromptTemplate
import json
from langchain_ollama.llms import OllamaLLM
from langchain.chains.llm import LLMChain
from langchain_core.runnables import RunnablePassthrough, RunnableLambda, RunnableParallel

In [2]:
# Subtopics chain

subtopics_template = PromptTemplate.from_template(
    """Generate a list of {number_of_subtopics} subheadings for a highschool essay on {topic}.
    Order the subheadings such that they have a logical sequence.
    Each subheading should be accompanied with a detailed discription on what the contents should be about.
    Provide output in json format as follows:

    {{'<heading1>':'<description1>',
    '<heading2>':'<description2>',...}}
    
    Only return the json object without any additional text."""
)
    
model = OllamaLLM(model="llama3.2:latest", run_local=True, format='json')

subtopic_chain = LLMChain(
    llm=model,
    prompt=subtopics_template,
    output_key="subtopics",
)

C:\Users\ssrim\AppData\Local\Temp\ipykernel_924\456135668.py:17: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  subtopic_chain = LLMChain(


In [3]:
# subtopic_chain.invoke(
#     {
#         "topic": "History of the Internet",
#         "number_of_subtopics": 3
#     }
# )

In [33]:
passthrough_chain = RunnableParallel().assign(
    subtopic_headings = lambda x: list(json.loads(x['subtopics']).keys()),
    subtopic_descriptions = lambda x: list(json.loads(x['subtopics']).values())
)

In [5]:
# passthrough_chain.invoke(subtopic_chain.invoke(
#     {
#         "topic": "History of the Internet",
#         "number_of_subtopics": 3
#     }
# ))

In [12]:
sequence_chain = subtopic_chain | passthrough_chain
result = sequence_chain.invoke(
    {
        "topic": "History of the Internet",
        "number_of_subtopics": 3
    }
)
result

{'topic': 'History of the Internet',
 'number_of_subtopics': 3,
 'subtopics': '{"Pre-History of the Internet": "This section will explore the origins and early development of the internet, including the key players, technologies, and events that laid the foundation for the modern internet. Topics may include the ARPANET project, the development of TCP/IP, and the role of government funding in the internet\'s early years.", \n"Evolution and Expansion": "This section will discuss the rapid expansion and evolution of the internet from the 1980s to the present day, including the growth of the World Wide Web, the rise of commercial internet service providers, and major technological advancements such as dial-up, broadband, and mobile networks.", \n"The Internet\'s Impact on Society": "This section will examine the significant impact of the internet on modern society, including its effects on communication, commerce, education, and culture. Topics may include the rise of e-commerce, the grow

In [27]:
def get_search_query(description):
    # print(description)
    search_query_prompt = PromptTemplate.from_template(
        """Generate a search query for the following description: {description}.
        The search query should be concise and relevant to the topic.
        Return only the search query without any additional text."""
    )
    model = OllamaLLM(model="llama3.2:latest", run_local=True)
    search_query_chain = LLMChain(
        llm=model,
        prompt=search_query_prompt,
        output_key="search_query",
    )
    return search_query_chain.invoke({"description": description})["search_query"]

In [28]:
search_query_chain = RunnablePassthrough.assign(
    search_queries=lambda x: RunnableLambda(get_search_query).batch(x['subtopic_descriptions'])
)

In [ ]:
sequence_chain = subtopic_chain | passthrough_chain | search_query_chain
result = sequence_chain.invoke(
    {
        "topic": "History of the Internet",
        "number_of_subtopics": 3
    }
)
result


{'topic': 'History of the Internet',
 'number_of_subtopics': 3,
 'subtopics': '{"Early Beginnings of the Internet":"This section will cover the early history of the internet, including the development of the ARPANET network in the 1960s and the creation of the first email by Ray Tomlinson in 1971.", \n"Expansion and Democratization": "The expansion of the internet to the general public began with the widespread adoption of dial-up internet in the 1990s, facilitated by companies like America Online (AOL) and CompuServe. This section will explore how the internet became more accessible to people from all walks of life.", \n"The Modern Internet: Social Media, Cloud Computing, and Beyond": "This final section will delve into the modern era of the internet, discussing the rise of social media platforms like Facebook and Twitter, the growth of cloud computing, and other emerging technologies that have transformed the way we communicate, work, and live."}',
 'subtopic_headings': ['Early Begin